In [61]:
import os
import autogen
from autogen import AssistantAgent, UserProxyAgent, ConversableAgent
from IPython import get_ipython

In [62]:
OAI_CONFIG_LIST = {
    'model': 'gpt-4o',
    'api_key': 'sk-'
}

In [63]:
OPENAI_MODEL = "gpt-4o"

config_list = [{
    "model": OPENAI_MODEL,
    "api_key": "sk-"
}]

In [64]:
llm_config = {
    "timeout": 600,
    "cache_seed": 42,
    "config_list": config_list,
    "temperature": 0,
    "functions": [
        {
            "name": "python",
            "description": "run cell in ipython and return the execution result.",
            "parameters": {
                "type": "object",
                "properties": {
                    "cell": {
                        "type": "string",
                        "description": "Valid Python cell to execute.",
                    }
                },
                "required": ["cell"],
            },
        }
    ],
    "config_list": config_list,
    "timeout": 120,
}


In [65]:
gpt_config = {
    "cache_seed": 42,
    "temperature": 0,
    "config_list": config_list,
    "timeout": 600,
}

In [66]:
# Schema Updater Agent
schema_gen = autogen.AssistantAgent(
    name="schema_gen",
    llm_config=llm_config,
    system_message=f"Specializes in reading court order documents iteratively and generating comprehensive schemas with all important information. Reads and updates schema for only 5 documents at a time, updating the schema after each batch. Schemas should be strictly based on the information in the documents. No dummy schema or hallucination allowed. Assign correct type to the keys based on the information in the documents, not your intuition.",
)

# Code Generator Agent
code_generator = autogen.AssistantAgent(
    name="code_generator",
    llm_config=llm_config,
    system_message="Generates necessary code for processing documents and managing schemas, returning it in the required format. Also executes the code. Strictly, only use python code generation."
)

# User Interface Agent
user_interface = autogen.UserProxyAgent(
   name="user_interface",
   system_message="Admin interface for user input. Supports user queries and termination signals. Invoke human feedback after each batch of schema generation and evaluation",
   is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
   code_execution_config={"work_dir":"coding"},
   max_consecutive_auto_reply=3,
   llm_config=llm_config,
   human_input_mode="ALWAYS"
)

closeness_checker = autogen.AssistantAgent(
    name="closeness_checker",
    llm_config=llm_config,
    system_message=(
         """Regenerates a random sample of 20 percent of original documents in a batch by just using the schema generated till then. It then compares the generated regenerated with the original documents. Evaluates regenerated court order documents to ensure all information from the original document is incorporated.
            By this comparison, the keys/questions not incorprated in the schema are identified and reported.
        Provides detailed feedback on the generated schema of the given batch to optimize the completeness of the schemas. It is strict critic of generated schema and flawlessly ensures that not even minute bit of information is lost. """
    )
)

In [67]:
def exec_python(cell):
    ipython = get_ipython()
    result = ipython.run_cell(cell)
    log = str(result.result)
    if result.error_before_exec is not None:
        log += f"\n{result.error_before_exec}"
    if result.error_in_exec is not None:
        log += f"\n{result.error_in_exec}"
    return log

def exec_sh(script):
    return user_interface.execute_code_blocks([("sh", script)])

In [68]:
schema_gen.register_function(function_map={
        "python": exec_python
})

code_generator.register_function(function_map={
        "python": exec_python
})
user_interface.register_function(function_map={
        "python": exec_python
})

closeness_checker.register_function(function_map={
        "python": exec_python
})

In [69]:
groupchat = autogen.GroupChat(agents=[user_interface, schema_gen,code_generator,closeness_checker], messages=[], max_round=100000)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt_config)

In [70]:
user_interface.initiate_chat(
    manager,
    message=""" Use Python only. Return the schema in standard JSON format. Load the text court orders from the provided folder path 'D:\ddd demo\court-judgement\judgements'. The files are inside the folder. First and foremost, read the text files for first batch.
    Read the court judgments iteratively, updating and improving the schema each time after reading a batch. Strictly perform document regeneration and closeness checking at the end updating schema for each batch by randomly selecting documents to regenerate and compare. Do not skip this step.
    Focus on generating a standardized schema with keys and their datatypes only, not values. Make it clear, I do not want to store any data from these documents but want to study the strcuture of these documents; that is, what are the questions that are needed to be answered to create document like these.
    """
)


user_interface (to chat_manager):

 Use Python only. Return the schema in standard JSON format. Load the text court orders from the provided folder path 'D:\ddd demo\court-judgement\judgements'. The files are inside the folder. First and foremost, read the text files for first batch.
    Read the court judgments iteratively, updating and improving the schema each time after reading a batch. Strictly perform document regeneration and closeness checking at the end updating schema for each batch by randomly selecting documents to regenerate and compare. Do not skip this step.
    Focus on generating a standardized schema with keys and their datatypes only, not values. Make it clear, I do not want to store any data from these documents but want to study the strcuture of these documents; that is, what are the questions that are needed to be answered to create document like these.
    

--------------------------------------------------------------------------------
schema_gen (to chat_manag

['Court Tehsildar-Sadar, Varanasi. Case number-J202414700104931 year-2024 under section-34/35 UP Revenue Code, Mauza-Bhagwanpur Pargana-Dehat Amanat, Tehsil-Sadar District-Varanasi\n\nSmt. Madhuri Devi vs. Smt. Sita Devi\n\nDecision\nThis suit of transfer is recorded on the registered sale deed number-1133 dated-16/02/2024 received from the registration office Varanasi. Which was instituted on the basis of the transfer report of the Lekhpal. Out of the area of \u200b\u200b0.3020 hectare of Arazi number-529mi, area of \u200b\u200b690 square feet i.e. area-64.12 square meters situated in Mauza-Bhagwanpur Pargana-Dehat Amanat, Tehsil and District-Varanasi has been sold by Smt. Sita Devi to Mrs. Madhuri Devi. As per rules, the suit was registered and a proclamation letter was issued which was served and the file is attached. In the absence of any objection, the suit is undisputed.\nThe buyer has stated in his evidence that even after purchasing the said land, as per the original sale deed,

schema_gen (to chat_manager):

***** Response from calling function "python" *****
['Court Tehsildar-Sadar, Varanasi. Case number-J202414700104931 year-2024 under section-34/35 UP Revenue Code, Mauza-Bhagwanpur Pargana-Dehat Amanat, Tehsil-Sadar District-Varanasi\n\nSmt. Madhuri Devi vs. Smt. Sita Devi\n\nDecision\nThis suit of transfer is recorded on the registered sale deed number-1133 dated-16/02/2024 received from the registration office Varanasi. Which was instituted on the basis of the transfer report of the Lekhpal. Out of the area of \u200b\u200b0.3020 hectare of Arazi number-529mi, area of \u200b\u200b690 square feet i.e. area-64.12 square meters situated in Mauza-Bhagwanpur Pargana-Dehat Amanat, Tehsil and District-Varanasi has been sold by Smt. Sita Devi to Mrs. Madhuri Devi. As per rules, the suit was registered and a proclamation letter was issued which was served and the file is attached. In the absence of any objection, the suit is undisputed.\nThe buyer has stated in hi

['Court of Tehsildar-Sadar, Varanasi. Case number-202414700105301 year-2024 under section-34/35 UP Revenue Code, Mauza-Nasirpur Pargana-Dehat Amanat, Tehsil-Sadar District-Varanasi\n\nSmt. Tanya Pal vs. Mrs. Sushma Yadav\n\nDecision\nThis suit of transfer is recorded on the registered sale deed number-1277 dated-20/02/2024 received from the registration office Varanasi. Which was instituted on the basis of the transfer report of the Lekhpal. Arazi number-Mi141/1 area 0.0490 hectare, out of the total purchased area of \u200b\u200b1700 sq. ft., the area sold of 1360 sq. ft. i.e. area-126.394 sq. m. situated in Mauza-Nasirpur Pargana-Dehat Amanat, Tehsil and District-Varanasi has been sold by Mrs. Sushma Yadav to Mrs. Tanya Pal. As per the rules, the suit was registered and proclamation letter was issued which was served and the file is attached. In the absence of any objection, the suit is undisputed.\nThe purchaser has stated in his evidence that even after purchasing the said land, the

closeness_checker (to chat_manager):

***** Response from calling function "python" *****
['Court of Tehsildar-Sadar, Varanasi. Case number-202414700105301 year-2024 under section-34/35 UP Revenue Code, Mauza-Nasirpur Pargana-Dehat Amanat, Tehsil-Sadar District-Varanasi\n\nSmt. Tanya Pal vs. Mrs. Sushma Yadav\n\nDecision\nThis suit of transfer is recorded on the registered sale deed number-1277 dated-20/02/2024 received from the registration office Varanasi. Which was instituted on the basis of the transfer report of the Lekhpal. Arazi number-Mi141/1 area 0.0490 hectare, out of the total purchased area of \u200b\u200b1700 sq. ft., the area sold of 1360 sq. ft. i.e. area-126.394 sq. m. situated in Mauza-Nasirpur Pargana-Dehat Amanat, Tehsil and District-Varanasi has been sold by Mrs. Sushma Yadav to Mrs. Tanya Pal. As per the rules, the suit was registered and proclamation letter was issued which was served and the file is attached. In the absence of any objection, the suit is undispute

...

code_generator (to chat_manager):

***** Response from calling function "python" *****
['Court of Tehsildar-Sadar, Varanasi. Case number-202414700101771 year-2024 under section-34/35 UP Revenue Code, Mauza-Phulwaria Pargana-Dehat Amanat, Tehsil-Sadar District-Varanasi\n\nSmt. Sushila Devi vs. Shri Vijay Kumar Pal\n\nDecision\nThis suit of transfer is mentioned on the registered donation letter number-57 dated-04/01/2024 received from the registration office Varanasi. Which was instituted on the basis of the transfer report of the Lekhpal. Mr. Vijay Kumar Pal has donated only 680 sq. ft. i.e. 63.19 sq. m. area out of his purchased area of \u200b\u200b1360 sq. ft. in Arazi number-Mr.456/1 area 0.6070 hectare to Mrs. Sushila Devi of Mauza-Phulwaria Pargana-Dehat Amanat, Tehsil and District-Varanasi. As per rules, the suit was registered and promulgation letter was issued which was served and the file is attached. In the absence of any objection, the suit is undisputed.\n\nThe donee has st

['\nCourt Tehsildar-Sadar, Varanasi. Case number-J202414700104026 year-2024 under section-34/35 UP Revenue Code, Mauza-Susuwahi Pargana-Dehat Amanat, Tehsil-Sadar District-Varanasi\n\nVaishnavi Construction vs Shri Sanjeev Kumar\n\nDecision\nThis suit of transfer is mentioned on the registered sale deed number-1003 dated-07/02/2024 received from the registration office Varanasi. Which was instituted on the basis of the transfer report of the Lekhpal. Out of Arazi number-435 area of \u200b\u200b0.4530 hectare, area of \u200b\u200b4342 sq. ft. i.e. area-403.53 sq. m. situated in Mauza-Susuwahi Pargana-Dehat Amanat, Tehsil and District-Varanasi, Shri Sanjeev Kumar has sold it to Bahk Vaishnavi Construction. As per rules, the suit was registered and promulgation letter was issued which was served and the file is attached. In absence of any objection, the suit is undisputed.\n\nThe buyer has stated in his evidence that even after purchasing the said land, as per the original sale deed, extr

schema_gen (to chat_manager):

***** Response from calling function "python" *****
['\nCourt Tehsildar-Sadar, Varanasi. Case number-J202414700104026 year-2024 under section-34/35 UP Revenue Code, Mauza-Susuwahi Pargana-Dehat Amanat, Tehsil-Sadar District-Varanasi\n\nVaishnavi Construction vs Shri Sanjeev Kumar\n\nDecision\nThis suit of transfer is mentioned on the registered sale deed number-1003 dated-07/02/2024 received from the registration office Varanasi. Which was instituted on the basis of the transfer report of the Lekhpal. Out of Arazi number-435 area of \u200b\u200b0.4530 hectare, area of \u200b\u200b4342 sq. ft. i.e. area-403.53 sq. m. situated in Mauza-Susuwahi Pargana-Dehat Amanat, Tehsil and District-Varanasi, Shri Sanjeev Kumar has sold it to Bahk Vaishnavi Construction. As per rules, the suit was registered and promulgation letter was issued which was served and the file is attached. In absence of any objection, the suit is undisputed.\n\nThe buyer has stated in his evi

['Court of Tehsildar-Sadar, Varanasi. Case number-202414700104593 year-2024 under section-34/35 UP Revenue Code, Mauza-Sarayanandan Pargana-Dehat Amanat, Tehsil-Sadar District-Varanasi\n\nSmt. Poonam Devi vs. Shri Naseem Shah\n\nDecision\nThis suit of transfer is mentioned on the registered sale deed received from the registration office Varanasi\n\nNo.-1227 dated-15/02/2024. Which was instituted on the basis of the transfer report of the Lekhpal. Out of Arazi number-36 area of \u200b\u200b0.2830 hectare, area of \u200b\u200b550.8 square feet i.e. area-51.189 square meters, the entire legal portion situated in Mauza-Sarayanandan Pargana-Dehat Amanat, Tehsil and District-Varanasi has been sold by Shri Naseem Shah to Smt. Poonam Devi. As per rules, the suit was registered and a proclamation letter was issued which was served and the file is attached. In the absence of any objection, the suit is undisputed.\nThe buyer has stated in his evidence that even after purchasing the said land, as

user_interface (to chat_manager):

***** Response from calling function "python" *****
['Court of Tehsildar-Sadar, Varanasi. Case number-202414700104593 year-2024 under section-34/35 UP Revenue Code, Mauza-Sarayanandan Pargana-Dehat Amanat, Tehsil-Sadar District-Varanasi\n\nSmt. Poonam Devi vs. Shri Naseem Shah\n\nDecision\nThis suit of transfer is mentioned on the registered sale deed received from the registration office Varanasi\n\nNo.-1227 dated-15/02/2024. Which was instituted on the basis of the transfer report of the Lekhpal. Out of Arazi number-36 area of \u200b\u200b0.2830 hectare, area of \u200b\u200b550.8 square feet i.e. area-51.189 square meters, the entire legal portion situated in Mauza-Sarayanandan Pargana-Dehat Amanat, Tehsil and District-Varanasi has been sold by Shri Naseem Shah to Smt. Poonam Devi. As per rules, the suit was registered and a proclamation letter was issued which was served and the file is attached. In the absence of any objection, the suit is undispu